## Semantic Similarity

In [2]:
import pandas as pd
import re

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import random

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GlobalAveragePooling1D,Lambda,Bidirectional
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.layers.embeddings import Embedding

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = pd.read_csv('train.csv')
 = pd.read_csv('test.csv')

C:\Users\singh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Text Preproprocessing

In [4]:
stops = set(stopwords.words('english'))

SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()
    
    def pad_str(s):
        return ' '+s+' '
    
    if pd.isnull(text):
        return ''

#    stops = set(stopwords.words("english"))
    # Clean the text, with the option to stem words.
    
    # Empty question
    
    if type(text) != str or text=='':
        return ''

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"can't", "cannot ", text)
    
    text = re.sub(r"doesn't", "does not ", text)
    text = re.sub(r"didn't", "did not ", text)
    text = re.sub(r"wouldn't", "would not ", text)
    text = re.sub(r"isn't", "is not ", text)
    
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", "  ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
    
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) # replace non-ascii word with special word
    
    # indian dollar
    
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)
    
    # clean text rules get from : https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text) 
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ", text, flags=re.IGNORECASE)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ", text, flags=re.IGNORECASE)  
    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE) 
    text = re.sub(r" actived ", " active ", text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ", text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ", text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ", text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ", text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ", text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ", text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" dna ", " DNA ", text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ", text)
    text = re.sub(r" banglore ", " Banglore ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    
    # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"
    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
    
    

    text = text.split()

    return text

In [5]:
with open("tokenization.pkl", 'rb') as file:
    tokenizer = pickle.load(file)
    
with open("embedding_matrix.pkl", 'rb') as file:
    embedding_matrix = pickle.load(file)

In [6]:
def text_processing(data):
    q1 = data[1]
    q2 = data[2]
    
    q1_transf = text_to_word_list(q1)
    q2_transf = text_to_word_list(q2)
    
    q1_token = tokenizer.texts_to_sequences(q1_transf)
    q2_token = tokenizer.texts_to_sequences(q2_transf)
    
    return pad_sequences([[j[0] for j in q1_token if len(j) != 0], [i[0] for i in q2_token if len(i) != 0]], maxlen=25) 

In [7]:
#defining embedding layer
nb_words = embedding_matrix.shape[0]                 

max_sentence_len = 25

embedding_layer = Embedding(nb_words,300,
                            weights=[embedding_matrix],
                            input_length=max_sentence_len,
                            trainable=False)
#dont train this layer!


#defining function to find similarity through vector distance 
def vec_distance(vects):
    x, y = vects
    return K.sum(K.square(x - y), axis=1, keepdims=True)


def vec_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [8]:
lstm_layer =LSTM(128)

sequence_1_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

distance=Lambda(vec_distance, output_shape=vec_output_shape)([x1, y1])

dense1 = Dense(16, activation='sigmoid')(distance)
dense1 = Dropout(0.3)(dense1)

bn2 = BatchNormalization()(dense1)

prediction=Dense(1, activation='sigmoid')(bn2)

model = Model(input=[sequence_1_input, sequence_2_input], output=prediction)

C:\Users\singh\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [9]:
model.load_weights('trained_model_ml_similarity.h5')

In [10]:
def prediction_semantic(i):
    data = test_df.iloc[i]
    data_new = text_processing(data)
    q1 = data_new[0].reshape(-1,25)
    q2 = data_new[1].reshape(-1,25)
    
    print('\nQuestion 1 -', data[1])
    
    print('\nQuestion 2 -', data[2])
    
    print("\nPrediction from the model ", 1*(model.predict([q1,q2])[0][0] >= 0.5))
    return None 

## Topic Modeling

In [11]:
with open("corpus.pkl", 'rb') as file:
    corpus = pickle.load(file)
    
with open("lda_model.pkl", 'rb') as file:
    lda_model = pickle.load(file)
    
with open("dfques.pkl", 'rb') as file:
    dfques = pickle.load(file)

In [12]:
def topics(i):
    index = i*2 + random.randint(1, 2)
    question = dfques[index]
    print("\nQuestion -", question)
    
    a = []
    b = []
    for i, j in lda_model[corpus[0]]:
        a.append(i)
        b.append(j)
    
    print("\nTopic distribution is -")
    print(pd.Series(b, index = ['Topic_1', 'Topic_2', 'Topic_3', 'Topic_4', 'Topic_5', 'Topic_6', 'Topic_7', 'Topic_8', 'Topic_9', 'Topic_10']))
    
    return None

In [17]:
test_df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


## Semantic Similarity Testing

In [37]:
i = int(input("Please enter question number (between 0 and 2345796) - "))

prediction_semantic(i)

Please enter question number (between 0 and 2345796) - 56

Question 1 - Which is a better watch brand Chopard vs Cartier?

Question 2 - Which watch brand has the most prestige?

Prediction from the model  0


## Topic Modeling Testing

In [38]:
i = int(input("Please enter question number - "))

topics(i)

Please enter question number - 67

Question - What is your review of Performance Testing?

Topic distribution is -
Topic_1     0.362430
Topic_2     0.033344
Topic_3     0.033339
Topic_4     0.370840
Topic_5     0.033344
Topic_6     0.033339
Topic_7     0.033339
Topic_8     0.033343
Topic_9     0.033340
Topic_10    0.033342
dtype: float64


In [39]:
lda_model.print_topics(10)

[(0,
  '0.015*"instagram" + 0.014*"bank" + 0.014*"food" + 0.012*"process" + 0.010*"considered" + 0.009*"air" + 0.009*"seen" + 0.009*"traffic" + 0.008*"battle" + 0.008*"videos"'),
 (1,
  '0.024*"facebook" + 0.013*"travel" + 0.012*"rid" + 0.011*"whatsapp" + 0.010*"school" + 0.010*"email" + 0.010*"body" + 0.009*"code" + 0.009*"fat" + 0.009*"delete"'),
 (2,
  '0.014*"student" + 0.012*"safe" + 0.011*"university" + 0.010*"tech" + 0.010*"engineer" + 0.009*"students" + 0.009*"mechanical" + 0.009*"police" + 0.009*"hotel" + 0.008*"major"'),
 (3,
  '0.025*"sex" + 0.019*"learning" + 0.013*"interview" + 0.012*"chinese" + 0.011*"video" + 0.011*"culture" + 0.010*"answers" + 0.010*"stop" + 0.010*"companies" + 0.009*"design"'),
 (4,
  '0.027*"ways" + 0.018*"system" + 0.017*"top" + 0.016*"earn" + 0.015*"mobile" + 0.011*"java" + 0.011*"interesting" + 0.009*"service" + 0.009*"type" + 0.009*"parents"'),
 (5,
  '0.019*"iphone" + 0.014*"2017" + 0.013*"place" + 0.011*"word" + 0.011*"class" + 0.011*"science" +